If you are not using GitHub [click here to open the notebook using nbviewer](http://nbviewer.jupyter.org/github/biocore/emperor/blob/new-api/examples/jackknifed.ipynb). **Note, if the plot below doesn't load when you run this notebook locally, make sure you click `Restart & Clear all Output` in the `Kernel` menu, then save the notebook and finally reload your browser. Otherwise the plot might not load correctly.**

This notebook creates a jackknifed PCoA plot based on multiple rarefactions.

In [1]:
%matplotlib inline
from emperor import Emperor, nbinstall

nbinstall()

from skbio.stats.ordination import pcoa
from skbio.diversity import beta_diversity

from biom import load_table

# pydata/scipy
import pandas as pd
import numpy as np

def load_mf(fn, index='#SampleID'):
    _df = pd.read_csv(fn, sep='\t', dtype=str, keep_default_na=False, na_values=[])
    _df.set_index(index, inplace=True)
    return _df

We are going to load data from [Fierer et al. 2010](http://www.pnas.org/content/107/14/6477.full) (the data was retrieved from study [232](https://qiita.ucsd.edu/study/description/232) in [Qiita](https://qiita.ucsd.edu), remember you need to be logged in to access the study).

In [2]:
mf = load_mf('keyboard/mapping-file.txt')
bt = load_table('keyboard/otu-table.biom')

Create 5 rarefactions and compute the Jaccard distance matrix for each resulting table.

In [3]:
ordinations = []
for r in range(5):
    rarefied = bt.subsample(1000)
    data = np.array([rarefied.data(i) for i in rarefied.ids()], dtype='int64')
    
    res = pcoa(beta_diversity('jaccard', data, rarefied.ids()))
    
    ordinations.append(res)

Jackknifed plots need a *master* set of coordinates where there rest of the matrices will be centered around.

In [4]:
master = ordinations[0]
jackknifed = ordinations[1:]

In [5]:
Emperor(master, mf, jackknifed=jackknifed)

/Users/yoshikivazquezbaeza/git_sw/emperor/emperor/util.py:743: EmperorWarning: Plots generated with `remote=True` using a development version of Emperor, may fail to load in the future (only the logo may be displayed instead of the plot). To avoid this, use a release version of Emperor.
  EmperorWarning)
